In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import re
import string
from collections import Counter
import nltk

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

import gensim

%matplotlib inline

In [2]:
df = pd.read_csv('df1csv.csv')
df.head()

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
0,OPERATIONAL,"{'lat': 20.6939009, 'lng': -105.2742384}",https://maps.gstatic.com/mapfiles/place_api/ic...,casa LAS PIÑAS,{'open_now': False},"[{'height': 640, 'html_attributions': ['<a hre...",ChIJIeVYvYNGIYQRh-9bPVK3fCU,"{'compound_code': 'MPVG+H8 Jarretaderas, Nayar...",4.6,ChIJIeVYvYNGIYQRh-9bPVK3fCU,...,casa LAS PIÑAS,4.6,"[{'author_name': 'Jovita Garcia Fernandez', 'a...",8.0,https://www.facebook.com/departamentos.laspinas/,"[5, 5, 5, 5, 5]","['An excellent place has all the comforts', 'I...",NaN,20.693901,-105.274238
1,OPERATIONAL,"{'lat': 20.6920731, 'lng': -105.2747303}",https://maps.gstatic.com/mapfiles/place_api/ic...,Villas Victoria Confort Jarretaderas,{'open_now': True},NaN,ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,"{'compound_code': 'MPRG+R4 Jarretaderas, Nayar...",5.0,ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,...,Villas Victoria Confort Jarretaderas,5.0,"[{'author_name': 'anahi chavez', 'author_url':...",1.0,https://www.villasvictoriacomfort.com/,[5],"['It was more than I expected! Cheap, spacious...",NaN,20.692073,-105.274730
2,OPERATIONAL,"{'lat': 20.6939068, 'lng': -105.2763239}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Posada Alejandra,NaN,"[{'height': 349, 'html_attributions': ['<a hre...",ChIJB2Yha4NGIYQRE-GTtF_VSAU,"{'compound_code': 'MPVF+HF Jarretaderas, Nayar...",4.0,ChIJB2Yha4NGIYQRE-GTtF_VSAU,...,Hotel Posada Alejandra,4.0,"[{'author_name': 'Juan Carlos Lopez', 'author_...",54.0,https://www.google.com/url?sa=t&source=web&rct...,"[3, 4, 3, 3, 5]","[""Just for one night it's alright"", 'Discreet'...",NaN,20.693907,-105.276324
3,OPERATIONAL,"{'lat': 20.6241471, 'lng': -105.2243218}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Azul Romero,{'open_now': True},"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJFW5JHF5FIYQRogsYTW-KvMI,"{'compound_code': 'JQFG+M7 Puerto Vallarta, Ja...",3.9,ChIJFW5JHF5FIYQRogsYTW-KvMI,...,Hotel Azul Romero,3.9,"[{'author_name': 'Al Reust', 'author_url': 'ht...",11.0,http://www.azulromero.com/,"[3, 4, 5, 4, 1]","['This is interesting, as it appears to be be ...",NaN,20.624147,-105.224322
4,OPERATIONAL,"{'lat': 20.6225929, 'lng': -105.2232518}",https://maps.gstatic.com/mapfiles/place_api/ic...,Vacational Apartment Apartamento Vacacional De...,NaN,"[{'height': 1052, 'html_attributions': ['<a hr...",ChIJ9321RV5FIYQRwnf0qJkeIqU,"{'compound_code': 'JQFG+2M Puerto Vallarta, Ja...",4.8,ChIJ9321RV5FIYQRwnf0qJkeIqU,...,Vacational Apartment Apartamento Vacacional De...,4.8,"[{'author_name': 'Daniel Chilldres', 'author_u...",11.0,https://www.airbnb.mx/rooms/23351610,"[5, 4, 5, 5, 5]","['I loved it here!', 'The place is clean, plea...",NaN,20.622593,-105.223252


In [3]:
df.iloc[6].all_reviews

"['Aight', 'Bad attitude of service offensive discrimination without giving reason or motive I definitely no longer recommend it', 'The net is very bad to rest because the man who takes care of the television at night makes the television very loud and does not let you sleep all morning.', 'An inexpensive place does not have the required amenities but is passable', 'Something good']"

In [4]:
remove_n = lambda x: re.sub('\\n',' ',x)
remove_singlen = lambda x: re.sub('\n',' ',x)
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['all_reviews'] = df.all_reviews.map(remove_n).map(punc_lower).map(alphanumeric)

In [5]:
df.iloc[3].all_reviews

'  this is interesting  as it appears to be be under new ownership  it has a new paint job on the front  i have not taken time to speak with the new managers  in the past few years i know of people who have did extended vacations as the prices were reasonable  access in the local community offers many things  i live across the street    there is a bus route to centro up the hill and return from centro down the hill  they have a few rooms for family or long term  i have seen buses unload and many people take over the hotel in the past  time will tell     comfortable and decent  it is   minutes from the beach nthe water heater does not work and neither does the internet  but good for spending the night    very good cows in guadalajara    very good    change of administration and they did not make the deposit valid according to why the past administration did not give it to them 😒 i had no other choice than to stay there since according to what i had booked and i had about   people obviou

In [6]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [7]:
remove_emoji = lambda x: emoji_pattern.sub(r'', x)

df['all_reviews'] = df.all_reviews.map(remove_emoji)

In [8]:
df.iloc[3].all_reviews

'  this is interesting  as it appears to be be under new ownership  it has a new paint job on the front  i have not taken time to speak with the new managers  in the past few years i know of people who have did extended vacations as the prices were reasonable  access in the local community offers many things  i live across the street    there is a bus route to centro up the hill and return from centro down the hill  they have a few rooms for family or long term  i have seen buses unload and many people take over the hotel in the past  time will tell     comfortable and decent  it is   minutes from the beach nthe water heater does not work and neither does the internet  but good for spending the night    very good cows in guadalajara    very good    change of administration and they did not make the deposit valid according to why the past administration did not give it to them  i had no other choice than to stay there since according to what i had booked and i had about   people obvious

In [9]:
df.iloc[28].all_reviews

'  jungle between the city  very exuberant place          '

In [10]:
df.iloc[14].all_reviews

'  good service and comfortable price                  '

In [11]:
# remove excessive spaces:
remove_spaces = lambda x: re.sub('        ',' ',x)
remove_spaces2 = lambda x: re.sub('   ', ' ', x)

df['all_reviews'] = df.all_reviews.map(remove_spaces).map(remove_spaces2)

In [12]:
df.iloc[28].all_reviews

'  jungle between the city  very exuberant place '

In [13]:
df[df['all_reviews'].str.len() <= 80]

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
12,OPERATIONAL,"{'lat': 20.6314507, 'lng': -105.2229678}",https://maps.gstatic.com/mapfiles/place_api/ic...,plaza central sur 150,NaN,NaN,ChIJzf0oz2NFIYQRuvlEAs6g2Fg,"{'compound_code': 'JQJG+HR Puerto Vallarta, Ja...",5.0,ChIJzf0oz2NFIYQRuvlEAs6g2Fg,...,plaza central sur 150,5.0,"[{'author_name': 'Sebastian Yañez', 'author_ur...",1.0,NaN,[5],quiet humble and very safe neighborhood,NaN,20.631451,-105.222968
14,OPERATIONAL,"{'lat': 20.6336142, 'lng': -105.2261243}",https://maps.gstatic.com/mapfiles/place_api/ic...,Apartamentos Lisboa,{'open_now': False},"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJT--Qj2RFIYQRCkuXi7C7zKI,"{'compound_code': 'JQMF+CH Puerto Vallarta, Ja...",5.0,ChIJT--Qj2RFIYQRCkuXi7C7zKI,...,Apartamentos Lisboa,5.0,"[{'author_name': 'Joseluis Avila', 'author_url...",6.0,NaN,"[5, 5, 5, 5, 5]",good service and comfortable price,NaN,20.633614,-105.226124
16,OPERATIONAL,"{'lat': 20.6387746, 'lng': -105.22502}",https://maps.gstatic.com/mapfiles/place_api/ic...,Depa vallarta,NaN,NaN,ChIJGeUor59FIYQRJ4VgrqfrkuU,"{'compound_code': 'JQQF+GX Puerto Vallarta, Ja...",5.0,ChIJGeUor59FIYQRJ4VgrqfrkuU,...,Depa vallarta,5.0,"[{'author_name': 'Mr Vallarta', 'author_url': ...",1.0,NaN,[5],without fear of life,NaN,20.638775,-105.225020
25,OPERATIONAL,"{'lat': 20.6550001, 'lng': -105.2198112}",https://maps.gstatic.com/mapfiles/place_api/ic...,Petra Glass Hostal,NaN,"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJMSEIsB1PIYQRSGnL5x7Bxi8,"{'compound_code': 'MQ4J+23 Puerto Vallarta, Ja...",5.0,ChIJMSEIsB1PIYQRSGnL5x7Bxi8,...,Petra Glass Hostal,5.0,"[{'author_name': 'Petra Glass', 'author_url': ...",1.0,NaN,[5],very clean and inexpensive,NaN,20.655000,-105.219811
28,OPERATIONAL,"{'lat': 20.6538149, 'lng': -105.2254394}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Río Cuale,NaN,NaN,ChIJZyLH6opFIYQRgw11fW6Zndw,"{'compound_code': 'MQ3F+GR Puerto Vallarta, Ja...",5.0,ChIJZyLH6opFIYQRgw11fW6Zndw,...,Hotel Río Cuale,5.0,"[{'author_name': 'Fernando Martiñon', 'author_...",2.0,NaN,"[5, 5]",jungle between the city very exuberant place,NaN,20.653815,-105.225439
29,OPERATIONAL,"{'lat': 20.6533935, 'lng': -105.2253539}",https://maps.gstatic.com/mapfiles/place_api/ic...,Nora,NaN,"[{'height': 632, 'html_attributions': ['<a hre...",ChIJVVVVVYFFIYQR5ZAO5E7UWkM,"{'compound_code': 'MQ3F+9V Puerto Vallarta, Ja...",4.2,ChIJVVVVVYFFIYQR5ZAO5E7UWkM,...,Nora,4.2,"[{'author_name': 'Jesus Reyna', 'author_url': ...",5.0,NaN,"[5, 5, 5, 1, 5]",ermozo place i loved it i love this place ...,NaN,20.653393,-105.225354
31,OPERATIONAL,"{'lat': 20.65340699999999, 'lng': -105.2253316}",https://maps.gstatic.com/mapfiles/place_api/ic...,Puerto Vallarta,NaN,"[{'height': 1280, 'html_attributions': ['<a hr...",ChIJJ7hW4RNFIYQReKCg-JvCd5w,"{'compound_code': 'MQ3F+9V Puerto Vallarta, Ja...",5.0,ChIJJ7hW4RNFIYQReKCg-JvCd5w,...,Puerto Vallarta,5.0,"[{'author_name': 'IXOTECUTLI ZIGEINER', 'autho...",1.0,NaN,[5],cool place pal shitter,NaN,20.653407,-105.225332
33,OPERATIONAL,"{'lat': 20.6597267, 'lng': -105.22686}",https://maps.gstatic.com/mapfiles/place_api/ic...,Iris,NaN,"[{'height': 2560, 'html_attributions': ['<a hr...",ChIJf23sZ49FIYQRE5ppLkSSQpE,"{'compound_code': 'MQ5F+V7 Puerto Vallarta, Ja...",4.7,ChIJf23sZ49FIYQRE5ppLkSSQpE,...,Iris,4.7,"[{'author_name': 'ANTONIO VILLALVAZO', 'author...",3.0,NaN,"[5, 5, 4]",super good,NaN,20.659727,-105.226860
46,OPERATIONAL,"{'lat': 20.6658674, 'lng': -105.2525124}",https://maps.gstatic.com/mapfiles/place_api/ic...,Condominio Suite Resort,NaN,"[{'height': 1944, 'html_attributions': ['<a hr...",ChIJ0WlxTrhFIYQRoKWZr6nJeaY,"{'compound_code': 'MP8W+8X Puerto Vallarta, Ja...",4.4,ChIJ0WlxTrhFIYQRoKWZr6nJeaY,...,Condominio Suite Resort,4.4,"[{'author_name': 'Rocio Benitez', 'author_url'...",5.0,NaN,"[5, 5, 3, 4, 5]",

In [14]:
df['all_reviews']

0        an excellent place has all the comforts  it ...
1        it was more than i expected  cheap  spacious...
2        just for one night it s alright  discreet  a...
3        this is interesting  as it appears to be be ...
4        i loved it here   the place is clean  pleasa...
                             ...                        
382      love this resort and hotel  best room ever  ...
383                                   really nice view  
384      new accommodations  but landscaping needs to...
385                                                cool 
386                   there was no beef the day i went  
Name: all_reviews, Length: 387, dtype: object

In [15]:
df.iloc[344].all_reviews

'  ok   '

In [16]:
df1 = df[df['all_reviews'].str.len() > 80]

In [17]:
df1.shape

(346, 26)

In [18]:
df1 = df1.reset_index(drop=True)
df1.tail()

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
341,OPERATIONAL,"{'lat': 20.6778627, 'lng': -105.2852463}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Tower 1 at Vidanta Nuevo Vallarta,{'open_now': True},"[{'height': 768, 'html_attributions': ['<a hre...",ChIJ4SJARyNEIYQRx5laF9dUiN0,"{'compound_code': 'MPH7+4W Nuevo Vallarta, Nay...",4.8,ChIJ4SJARyNEIYQRx5laF9dUiN0,...,Grand Luxxe Tower 1 at Vidanta Nuevo Vallarta,4.8,"[{'author_name': 'Diane Tillman', 'author_url'...",371.0,NaN,"[5, 5, 3, 5, 5]",great location to pools bar restaurants m...,NaN,20.677863,-105.285246
342,OPERATIONAL,"{'lat': 20.6851287, 'lng': -105.287574}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grupo Vidanta Headquarters,NaN,"[{'height': 1001, 'html_attributions': ['<a hr...",ChIJK_fqDKNBIYQRU-pYVJ--ZNI,"{'compound_code': 'MPP6+3X Nuevo Vallarta, Nay...",3.8,ChIJK_fqDKNBIYQRU-pYVJ--ZNI,...,Grupo Vidanta Headquarters,3.8,"[{'author_name': 'Elsy B', 'author_url': 'http...",29.0,https://www.vidanta.com/,"[1, 5, 5, 2, 5]",very disappointed it seems that every time ...,NaN,20.685129,-105.287574
343,OPERATIONAL,"{'lat': 20.6828908, 'lng': -105.2788986}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Tower 5,NaN,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJI9D-5CVEIYQRyClZqRFcuoM,"{'compound_code': 'MPMC+5C Nuevo Vallarta, Nay...",4.8,ChIJI9D-5CVEIYQRyClZqRFcuoM,...,Grand Luxxe Tower 5,4.8,"[{'author_name': 'Mary Mackey', 'author_url': ...",437.0,https://www.vidanta.com/web/nuevo-vallarta,"[5, 5, 5, 5, 5]",the rooms are opulent the roof top pool and...,NaN,20.682891,-105.278899
344,OPERATIONAL,"{'lat': 20.6769383, 'lng': -105.285772}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Punta Tower at Vidanta Nuevo Vallarta,{'open_now': True},"[{'height': 3480, 'html_attributions': ['<a hr...",ChIJ7zhNNCNEIYQRDyRrWYr8hLk,"{'compound_code': 'MPG7+QM Nuevo Vallarta, Nay...",4.9,ChIJ7zhNNCNEIYQRDyRrWYr8hLk,...,Grand Luxxe Punta Tower at Vidanta Nuevo Vallarta,4.9,"[{'author_name': 'John Bittle', 'author_url': ...",64.0,https://www.vidanta.com/en/web/nuevo-vallarta/...,"[5, 5, 5, 5, 5]",love this resort and hotel best room ever ...,NaN,20.676938,-105.285772
345,OPERATIONAL,"{'lat': 20.6843008, 'lng': -105.2780858}",https://maps.gstatic.com/mapfiles/place_api/ic...,Torre 6 The Grand Luxxe,NaN,"[{'height': 2992, 'html_attributions': ['<a hr...",ChIJo9we6EJFIYQRx9y4RoVO4Bs,"{'compound_code': 'MPMC+PQ Jarretaderas, Nayar...",4.6,ChIJo9we6EJFIYQRx9y4RoVO4Bs,...,Torre 6 The Grand Luxxe,4.6,"[{'author_name': 'Dana Nyborg', 'author_url': ...",15.0,NaN,"[4, 5, 3, 5, 5]",new accommodations but landscaping needs to...,NaN,20.684301,-105.278086


In [19]:
df1.describe(include='all')

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
count,346,346,346,346,176,336,346,345,346.000000,346,...,346,346.000000,346,346.000000,256,346,346,1.0,346.000000,346.000000
unique,1,345,4,343,2,336,346,338,NaN,346,...,343,NaN,346,NaN,250,193,346,NaN,NaN,NaN
top,OPERATIONAL,"{'lat': 20.65340699999999, 'lng': -105.2253316}",https://maps.gstatic.com/mapfiles/place_api/ic...,Villa Azul Celeste,{'open_now': True},"[{'height': 2272, 'html_attributions': ['<a hr...",ChIJDyiBVP5aIYQRrlhVOToa43s,"{'compound_code': 'HQM4+WF Puerto Vallarta, Ja...",NaN,ChIJDyiBVP5aIYQRrlhVOToa43s,...,Villa Azul Celeste,NaN,"[{'author_name': 'Jorge Alvarez', 'author_url'...",NaN,https://www.vidanta.com/web/nuevo-vallarta,"[5, 5, 5, 5, 5]",always nice make sure you have solid commun...,NaN,NaN,NaN
freq,346,2,324,2,136,1,1,2,NaN,1,...,2,NaN,1,NaN,3,65,1,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.389595,NaN,...,NaN,4.388439,NaN,412.187861,NaN,NaN,NaN,2.0,20.620304,-105.241177
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.499050,NaN,...,NaN,0.498443,NaN,899.480495,NaN,NaN,NaN,NaN,0.034150,0.016527
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,...,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,2.0,20.532667,-105.291951
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.200000,NaN,...,NaN,4.200000,NaN,11.000000,NaN,NaN,NaN,2.0,20.600183,-105.246052
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,NaN,...,NaN,4.500000,NaN,46.000000,NaN,NaN,NaN,2.0,20.614583,-105.236044
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.700000,NaN,...,NaN,4.700000,NaN,286.000000,NaN,NaN,NaN,2.0,20.643411,-105.231297


In [20]:
df1[df1['rating_x']<=3]

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
57,OPERATIONAL,"{'lat': 20.6590553, 'lng': -105.2469}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Villa Varadero,NaN,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJOdpzlbxFIYQRI-mzY60HRcQ,"{'compound_code': 'MQ53+J6 Puerto Vallarta, Ja...",2.7,ChIJOdpzlbxFIYQRI-mzY60HRcQ,...,Hotel Villa Varadero,2.7,"[{'author_name': 'xaviere pedroxa lopex', 'aut...",3.0,NaN,"[1, 4, 3]",there is an error a maps guides you to a v...,NaN,20.659055,-105.246900
130,OPERATIONAL,"{'lat': 20.6263699, 'lng': -105.2281492}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Vallarta,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ093DB2dFIYQRYxq91vgbkJw,"{'compound_code': 'JQGC+GP Puerto Vallarta, Ja...",2.6,ChIJ093DB2dFIYQRYxq91vgbkJw,...,Hotel Vallarta,2.6,"[{'author_name': 'Kimberly Tyner', 'author_url...",7.0,NaN,"[1, 3, 1, 3, 4]",cruel people good place to rest only that t...,NaN,20.626370,-105.228149
142,OPERATIONAL,"{'lat': 20.613498, 'lng': -105.2328524}",https://maps.gstatic.com/mapfiles/place_api/ic...,The hostal spot,NaN,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJ28z2UUVFIYQRiW3MK5XgGnc,"{'compound_code': 'JQ78+9V Puerto Vallarta, Ja...",2.6,ChIJ28z2UUVFIYQRiW3MK5XgGnc,...,The hostal spot,2.6,"[{'author_name': 'Karen gudgel', 'author_url':...",7.0,NaN,"[1, 5, 5, 1, 1]",no hot water and the electricity was only a ...,NaN,20.613498,-105.232852
158,OPERATIONAL,"{'lat': 20.6131722, 'lng': -105.232317}",https://maps.gstatic.com/mapfiles/place_api/ic...,Departamento en el corazón de Vallarta,NaN,"[{'height': 2592, 'html_attributions': ['<a hr...",ChIJA7LxQ0VFIYQRiylo05bpyrM,"{'compound_code': 'JQ79+73 Puerto Vallarta, Ja...",1.0,ChIJA7LxQ0VFIYQRiylo05bpyrM,...,Departamento en el corazón de Vallarta,1.0,"[{'author_name': 'Addy Munguia', 'author_url':...",2.0,NaN,"[1, 1]",the business has been closed for years and g...,NaN,20.613172,-105.232317
279,OPERATIONAL,"{'lat': 20.5916992, 'lng': -105.24157}",https://maps.gstatic.com/mapfiles/place_api/ic...,Casa Destination Of Health Resort,NaN,"[{'height': 4128, 'html_attributions': ['<a hr...",ChIJrZeSN7ZaIYQRvhiC-HiH6lw,"{'compound_code': 'HQR5+M9 Puerto Vallarta, Ja...",1.8,ChIJrZeSN7ZaIYQRvhiC-HiH6lw,...,Casa Destination Of Health Resort,1.8,"[{'author_name': 'Bianca Ferratt', 'author_url...",4.0,http://www.casadestinationofhealth.mxhotel.site/,"[1, 1, 4, 1]",i made a reservation and the address does no...,NaN,20.591699,-105.241570
313,OPERATIONAL,"{'lat': 20.5483213, 'lng': -105.2731479}",https://maps.gstatic.com/mapfiles/place_api/ic...,Gran park royal,NaN,"[{'height': 943, 'html_attributions': ['<a hre...",ChIJVSghLVhbIYQRd69si_Ia1-U,"{'compound_code': 'GPXG+8P Aguacate, Jalisco, ...",2.7,ChIJVSghLVhbIYQRd69si_Ia1-U,...,Gran park royal,2.7,"[{'author_name': 'Kevin MX', 'author_url': 'ht...",3.0,NaN,"[4, 1, 3]",nice hotel good food pool is cold kids po...,NaN,20.548321,-105.273148


In [21]:
# df1.to_csv(r'hotels_cleaned.csv', index=True, header=True, index_label=False)

In [22]:
df1 = df1[df1['rating_x']>3.5]

In [23]:
df1.shape

(329, 26)

# Word Embeddings

# Pre-trained Word2Vect Model

In [24]:
import os

# Setup nltk corpora path and Google Word2Vec location
google_vec_file = '/Users/lindsayread/Downloads/GoogleNews-vectors-negative300.bin'

In [25]:
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)



In [26]:
def word_vectorizer(document):
    stoplist = set('for a an it its very of the and to in'.split())
    texts = [word for word in document.lower().split() if word not in stoplist]
    
    return texts

In [27]:
initial_df = df1.all_reviews.apply(word_vectorizer)
initial_df.head()

0    [excellent, place, has, all, comforts, is, won...
1    [was, more, than, i, expected, cheap, spacious...
2    [just, one, night, s, alright, discreet, simpl...
3    [this, is, interesting, as, appears, be, be, u...
4    [i, loved, here, place, is, clean, pleasant, e...
Name: all_reviews, dtype: object

In [29]:
def viable_words(words):
    viable_words = []
    for word in words:
        try:
            model[f'{word}']
            viable_words.append(word)
        except:
            pass
    return viable_words

In [30]:
viables_df = initial_df.apply(viable_words)

In [ ]:
# def mean_hotel_vector(hotel):

#     df_first_word = pd.DataFrame([model[hotel[0]]])
#     for word in df_first_word[1:]:
#         df_first_word = pd.concat([df_first_word, pd.DataFrame([model[f'{word}']])], ignore_index=True)
#     first_hotel = pd.DataFrame(df_first_word.mean())
#     return(first_hotel)

In [ ]:
# df_all_vectors = df_vector_first
# for hotel in first_five:
#     first_word = pd.DataFrame([model[hotel[0]]])
#     for word in hotel[1:]:
#         first_word = pd.concat([df_first, pd.DataFrame([model[f'{word}']])], ignore_index=True)
#     first_word = pd.DataFrame(first_word.mean())
#     df_all_vectors = pd.concat([df_all_vectors, first_word], axis=1, ignore_index=True)
#     #print(hotel[1])
# df_all_vectors

In [31]:
def mapped_vectors(viable_words):
    df_first = pd.DataFrame([model[viable_words[0]]])
    for word in viable_words[1:]:
        df_first = pd.concat([df_first, pd.DataFrame([model[f'{word}']])], ignore_index=True)
    df_mean = pd.DataFrame(df_first.mean())
    return df_mean.T

In [33]:
first_hotel = viables_df.iloc[1]

In [37]:
df_initial = mapped_vectors(first_hotel)
df_initial

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.019035,0.029907,0.059304,0.10355,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651


In [38]:
for hotel in viables_df:
    df_initial = pd.concat([df_initial, mapped_vectors(hotel)])
df_initial

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.019035,0.029907,0.059304,0.103550,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651
0,-0.020985,0.079720,0.014596,0.039373,-0.042453,0.007990,0.044106,-0.109730,0.002894,0.106301,...,-0.139549,-0.022568,-0.140958,-0.000347,-0.032293,0.057663,0.041021,-0.022186,0.039479,0.017561
0,-0.019035,0.029907,0.059304,0.103550,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651
0,0.017837,0.036630,-0.004139,0.061884,-0.056658,0.002148,0.037810,-0.086593,0.089868,0.102893,...,-0.061085,0.071887,-0.081440,0.037477,-0.026248,0.009870,0.001176,-0.022683,0.005633,-0.028478
0,0.013963,0.004875,0.019757,0.093845,-0.042801,-0.014613,0.018288,-0.080157,0.095798,0.067508,...,-0.061187,0.057206,-0.080036,0.038635,-0.006875,0.017680,-0.004359,-0.033175,0.026842,-0.050020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-0.003025,0.022434,0.040814,0.123567,-0.031979,-0.005519,0.031609,-0.105080,0.030516,0.080158,...,-0.087490,0.035833,-0.124875,0.068512,-0.005762,-0.012329,-0.001398,-0.049006,0.060797,-0.031703
0,0.031462,0.045366,0.021865,0.096295,-0.054402,-0.004726,0.026638,-0.063134,0.036097,0.058590,...,-0.064844,0.045405,-0.084736,0.050469,-0.016660,0.014252,-0.006320,-0.035374,0.054294,-0.022848
0,-0.029009,0.031826,0.042055,0.063677,-0.046213,-0.030179,0.058998,-0.075694,0.055920,0.084873,...,-0.051368,0.025235,-0.085530,0.051253,-0.005745,-0.037851,0.050975,-0.062034,0.011584,-0.012479
0,-0.028351,0.062371,0.050773,0.069491,-0.044939,-0.024586,0.040510,-0.126087,0.084763,0.052136,...,-0.055389,0.019512,-0.071179,0.039368,-0.036664,-0.087397,-0.031105,-0.027344,0.000923,-0.018604


In [42]:
reindexed = df_initial.reset_index(drop=True)
reindexed.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.019035,0.029907,0.059304,0.103550,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651
1,-0.020985,0.079720,0.014596,0.039373,-0.042453,0.007990,0.044106,-0.109730,0.002894,0.106301,...,-0.139549,-0.022568,-0.140958,-0.000347,-0.032293,0.057663,0.041021,-0.022186,0.039479,0.017561
2,-0.019035,0.029907,0.059304,0.103550,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651
3,0.017837,0.036630,-0.004139,0.061884,-0.056658,0.002148,0.037810,-0.086593,0.089868,0.102893,...,-0.061085,0.071887,-0.081440,0.037477,-0.026248,0.009870,0.001176,-0.022683,0.005633,-0.028478
4,0.013963,0.004875,0.019757,0.093845,-0.042801,-0.014613,0.018288,-0.080157,0.095798,0.067508,...,-0.061187,0.057206,-0.080036,0.038635,-0.006875,0.017680,-0.004359,-0.033175,0.026842,-0.050020


In [44]:
reindexed.shape

(330, 300)

In [45]:
full_df = reindexed.drop([0])
full_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1,-0.020985,0.079720,0.014596,0.039373,-0.042453,0.007990,0.044106,-0.109730,0.002894,0.106301,...,-0.139549,-0.022568,-0.140958,-0.000347,-0.032293,0.057663,0.041021,-0.022186,0.039479,0.017561
2,-0.019035,0.029907,0.059304,0.103550,0.005379,0.023773,0.008163,-0.111519,0.104652,0.082924,...,-0.104172,0.053316,-0.093506,0.020936,0.036129,-0.052326,-0.004178,-0.034481,0.027336,-0.041651
3,0.017837,0.036630,-0.004139,0.061884,-0.056658,0.002148,0.037810,-0.086593,0.089868,0.102893,...,-0.061085,0.071887,-0.081440,0.037477,-0.026248,0.009870,0.001176,-0.022683,0.005633,-0.028478
4,0.013963,0.004875,0.019757,0.093845,-0.042801,-0.014613,0.018288,-0.080157,0.095798,0.067508,...,-0.061187,0.057206,-0.080036,0.038635,-0.006875,0.017680,-0.004359,-0.033175,0.026842,-0.050020
5,-0.003060,0.020569,0.042245,0.044713,-0.046801,0.011214,0.026570,-0.080710,0.059676,0.059297,...,-0.110012,0.028893,-0.101961,0.053882,-0.002300,0.041499,-0.004092,-0.040666,0.030417,-0.018074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,-0.003025,0.022434,0.040814,0.123567,-0.031979,-0.005519,0.031609,-0.105080,0.030516,0.080158,...,-0.087490,0.035833,-0.124875,0.068512,-0.005762,-0.012329,-0.001398,-0.049006,0.060797,-0.031703
326,0.031462,0.045366,0.021865,0.096295,-0.054402,-0.004726,0.026638,-0.063134,0.036097,0.058590,...,-0.064844,0.045405,-0.084736,0.050469,-0.016660,0.014252,-0.006320,-0.035374,0.054294,-0.022848
327,-0.029009,0.031826,0.042055,0.063677,-0.046213,-0.030179,0.058998,-0.075694,0.055920,0.084873,...,-0.051368,0.025235,-0.085530,0.051253,-0.005745,-0.037851,0.050975,-0.062034,0.011584,-0.012479
328,-0.028351,0.062371,0.050773,0.069491,-0.044939,-0.024586,0.040510,-0.126087,0.084763,0.052136,...,-0.055389,0.019512,-0.071179,0.039368,-0.036664,-0.087397,-0.031105,-0.027344,0.000923,-0.018604


In [47]:
full_df.to_csv(r'nlp_vectors.csv', index=True, header=True, index_label=False)

In [46]:
pd.DataFrame(cosine_similarity(full_df))

,0,1,2,3,4,5,6,7,8,9,...,319,320,321,322,323,324,325,326,327,328
0,1.000000,0.590262,0.760175,0.696436,0.839844,0.773283,0.684834,0.781843,0.716060,0.804452,...,0.713446,0.686826,0.646678,0.777756,0.735761,0.768725,0.764621,0.746810,0.740613,0.744490
1,0.590262,1.000000,0.725514,0.711866,0.717221,0.695812,0.684057,0.742357,0.700342,0.704514,...,0.718918,0.752776,0.673115,0.710607,0.765505,0.705583,0.721169,0.747036,0.661576,0.707096
2,0.760175,0.725514,1.000000,0.840045,0.892312,0.841961,0.842499,0.890320,0.807070,0.880888,...,0.831503,0.763239,0.802634,0.811318,0.838234,0.814166,0.840353,0.756447,0.741766,0.771834
3,0.696436,0.711866,0.840045,1.000000,0.860659,0.827249,0.897218,0.907629,0.838324,0.816952,...,0.814287,0.819810,0.914822,0.819653,0.882776,0.854925,0.933905,0.734911,0.719738,0.796822
4,0.839844,0.717221,0.892312,0.860659,1.000000,0.848623,0.843288,0.914028,0.861967,0.892854,...,0.836515,0.827888,0.812050,0.823108,0.870822,0.863316,0.881807,0.818916,0.763138,0.833014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,0.768725,0.705583,0.814166,0.854925,0.863316,0.864587,0.793570,0.877115,0.828389,0.797395,...,0.881387,0.868273,0.829362,0.896326,0.895056,1.000000,0.904188,0.843957,0.830970,0.841292
325,0.764621,0.721169,0.840353,0.933905,0.881807,0.890402,0.867587,0.910343,0.807560,0.852888,...,0.856728,0.840407,0.918340,0.863244,0.893900,0.904188,1.000000,0.776059,0.788969,0.818146
326,0.746810,0.747036,0.756447,0.734911,0.818916,0.803915,0.676853,0.818269,0.805240,0.784602,...,0.834896,0.893005,0.704646,0.876438,0.846843,0.843957,0.776059,1.000000,0.814182,0.862721
327,0.740613,0.661576,0.741766,0.719738,0.763138,0.808399,0.692630,0.808448,0.711916,0.788065,...,0.814504,0.835060,0.719060,0.864359,0.820170,0.830970,0.788969,0.814182,1.000000,0.796572
